In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt


Traceback (most recent call last):
  File "c:\Users\steph\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1
    %matplotlib inline
    ^
SyntaxError: invalid syntax



In [ ]:
import numpy as np
import pandas as pd
import datetime as dt


# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


In [ ]:
# Create engine using the `hawaii.sqlite` database file
engine = create_engine("sqlite:///Resources/hawaii.sqlite")


In [ ]:
# Declare a Base using `automap_base()`
Base=automap_base()
# Use the Base class to reflect the database tables
Base.prepare(engine,reflect=True)


C:\Users\steph\AppData\Local\Temp\ipykernel_26900\3957317230.py:4: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine,reflect=True)


In [ ]:
# Print all of the classes mapped to the Base
for cls in Base.classes:
    print(cls.__table__.name)

station
measurement


In [ ]:
# Assign the measurement class to a variable called `Measurement` and
# the station class to a variable called `Station`
Measurement=Base.classes['measurement']
Station=Base.classes['station']

In [ ]:
# Create a session
session=Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
#most_recent=session.query(Measurement).order_by(Measurement.date.desc()).first()
start_point=session.query(func.max(Measurement.date)).first()[0]

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results.
# Starting from the most recent data point in the database.

# Calculate the date one year from the last date in data set.
# Perform a query to retrieve the data and precipitation scores
last_year=dt.date(2017,8,23)-dt.timedelta(days=365)
data=session.query(Measurement.prcp,Measurement.date).filter(Measurement.date>last_year)
#data=data.group_by(Measurement.date).with_entities(func.max(Measurement.prcp)).all()
# Save the query results as a Pandas DataFrame. Explicitly set the column names
df=pd.DataFrame(data)
df=df.rename(columns={'prcp':'precipitation'})
# Sort the dataframe by date
df=df.sort_values('date')
# Use Pandas Plotting with Matplotlib to plot the data
df.plot(x='date',y='precipitation',rot=90)
plt.ylabel("Inches")
plt.show()


Traceback (most recent call last):
  File "c:\Users\steph\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 16, in <module>
NameError: name 'plt' is not defined



In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
stats=df['precipitation'].describe()
print(stats)

count    2015.000000
mean        0.176462
std         0.460288
min         0.000000
25%         0.000000
50%         0.020000
75%         0.130000
max         6.700000
Name: precipitation, dtype: float64


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
count=session.query(Station).count()

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
results=session.query(Measurement.station,func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
results

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
station_id=results[0][0]
results=session.query(func.min(Measurement.tobs),func.max(Measurement.tobs),func.avg(Measurement.tobs)).filter(Measurement.station==station_id).all()
print(results)

[(54.0, 85.0, 71.66378066378067)]


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results=session.query(Measurement).filter(Measurement.station==station_id).filter(Measurement.date>dt.datetime.strptime(most_recent,'%Y-%m-%d')-dt.timedelta(days=365)).with_entities(Measurement.tobs)
df=pd.DataFrame(results)
df.plot(kind='hist',bins=12)
plt.show()

Traceback (most recent call last):
  File "c:\Users\steph\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 3, in <module>
NameError: name 'most_recent' is not defined



# Close Session

In [ ]:
# Close Session
session.close()
